# Logic: `logic.py`; Chapters 6-8

This notebook describes the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module, which covers Chapters 6 (Logical Agents),  7 (First-Order Logic) and  8 (Inference in First-Order Logic) of *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

We'll start by looking at `Expr`, the data type for logical sentences, and the convenience function `expr`. We'll be covering two types of knowledge bases, `PropKB` - Propositional logic knowledge base and `FolKB` - First order logic knowledge base. We will construct a propositional knowledge base of a specific situation in the Wumpus World. We will next go through the `tt_entails` function and experiment with it a bit. The `pl_resolution` and `pl_fc_entails` functions will come next. We'll study forward chaining and backward chaining algorithms for `FolKB` and use them on `crime_kb` knowledge base.

But the first step is to load the code:

In [1]:
from utils import *
from logic import *
from notebook import psource

## Logical Sentences

The `Expr` class is designed to represent any kind of mathematical expression. The simplest type of `Expr` is a symbol, which can be defined with the function `Symbol`:

In [2]:
Symbol('x')

x

Or we can define multiple symbols at the same time with the function `symbols`:

In [3]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

We can combine `Expr`s with the regular Python infix and prefix operators. Here's how we would form the logical sentence "P and not Q":

In [4]:
P & ~Q

(P & ~Q)

This works because the `Expr` class overloads the `&` operator with this definition:

```python
def __and__(self, other): return Expr('&',  self, other)```
     
and does similar overloads for the other operators. An `Expr` has two fields: `op` for the operator, which is always a string, and `args` for the arguments, which is a tuple of 0 or more expressions. By "expression," I mean either an instance of `Expr`, or a number. Let's take a look at the fields for some `Expr` examples:

In [5]:
sentence = P & ~Q

sentence.op

'&'

In [6]:
sentence.args

(P, ~Q)

In [7]:
P.op

'P'

In [8]:
P.args

()

In [9]:
Pxy = P(x, y)

Pxy.op

'P'

In [10]:
Pxy.args

(x, y)

It is important to note that the `Expr` class does not define the *logic* of Propositional Logic sentences; it just gives you a way to *represent* expressions. Think of an `Expr` as an [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree).  Each of the `args` in an `Expr` can be either a symbol, a number, or a nested `Expr`. We can nest these trees to any depth. Here is a deply nested `Expr`:

In [11]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operators for Constructing Logical Sentences

Here is a table of the operators that can be used to form sentences. Note that we have a problem: we want to use Python operators to make sentences, so that our programs (and our interactive sessions like the one here) will show simple code. But Python does not allow implication arrows as operators, so for now we have to use a more verbose notation that Python does allow: `|'==>'|` instead of just `==>`. Alternately, you can always use the more verbose `Expr` constructor forms:

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

Here's an example of defining a sentence with an implication arrow:

In [12]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

### Animal Classification

In [13]:

clauses = []

### I1. If the animal has hair then it is a mammal
clauses.append(expr("(Hair(x) ==> Mammal(x))"))

### I2. If the animal gives milk then it is a mammal
clauses.append(expr("(GivesMilk(x) ==> Mammal(x))"))

### I3. If the animal has feathers then it is a bird
clauses.append(expr("(HasFeathers(x) ==> Bird(x))"))

### I4. If the animal flies and it lays eggs then it is a bird
clauses.append(expr("(Flies(x) & LaysEggs(x) ==> Bird(x))"))

### I5. If the animal is a mammal and it eats meat then it is a carnivore
clauses.append(expr("(Mammal(x) & EatsMeat(x) ==> Carnivore(x))"))

### I6. If the animal is a mammal and it has pointed teeth and it has claws and its eyes point forward
###   then it is a carnivore
clauses.append(expr("(Mammal(x) & PointedTeeth(x) & HasClaws(x) & EyesPointForward(x) ==> Carnivore(x))"))

### I7. If the animal is a mammal and it has hoofs then it is an ungulate
clauses.append(expr("(Mammal(x) & HasHoofs(x) ==> Ungulate(x))"))

### 18. If the animal is a mammal and it chews cud then it is an ungulate
clauses.append(expr("(Mammal(x) & ChewsCud(x) ==> Ungulate(x))"))

### I9. If the animal is a carnivore and it has a tawny color and it has dark spots then it is a cheetah
clauses.append(expr("(Carnivore(x) & TawnyColor(x) & DarkSpots(x) ==> Cheetah(x))"))

### I10. If the animal is a carnivore and it has a tawny color and it has black stripes then it is a tiger
clauses.append(expr("(Carnivore(x) & TawnyColor(x) & BlackStripes(x) ==> Tiger(x))"))

### I11. If the animal is an ungulate and it has long legs and it has a long neck and it has a tawny color
###   and it has dark spots then it is a giraffe
clauses.append(expr("(Ungulate(x) & TawnyColor(x) & LongLegs(x) & LongNeck(x) & DarkSpots(x) ==> Giraffe(x))"))

### I12. If the animal is an ungulate and it has a white color and it has black stripes then it is a zebra
clauses.append(expr("(Ungulate(x) & WhiteColor(x) & BlackStripes(x) ==> Zebra(x))"))

### Il3. If the animal is a bird and it does not fly and it has long legs and it has a long neck and it is black
###   and white then it is an ostrich,
clauses.append(expr("(Bird(x) & DoesNotFly(x) & LongLegs(x) & LongNeck(x) & BlackAndWhite(x) ==> Ostrich(x))"))

### Il4. If the animal is a bird and it does not fly and it swims and it is black and white then it is a
###    penguin
clauses.append(expr("(Bird(x) & DoesNotFly(x) & Swims(x) & BlackAndWhite(x) ==> Penguin(x))"))

### Il5. If the animal is a bird and it is a good flyer then it is an albatross.
clauses.append(expr("(Bird(x) & GoodFlyer(x) ==> Albatross(x))"))

animals_kb = FolKB(clauses)
print("done")

done


In [14]:
animals_kb.tell(expr("GivesMilk(Animal1)"))
animals_kb.tell(expr("HasHoofs(Animal1)"))
animals_kb.tell(expr("EatsMeat(Animal1)"))
animals_kb.tell(expr("TawnyColor(Animal1)"))
animals_kb.tell(expr("BlackStripes(Animal1)"))

animals_kb.tell(expr("GivesMilk(Animal2)"))
animals_kb.tell(expr("HasHoofs(Animal2)"))
animals_kb.tell(expr("EatsMeat(Animal2)"))
animals_kb.tell(expr("WhiteColor(Animal2)"))
animals_kb.tell(expr("BlackStripes(Animal2)"))

animals_kb.tell(expr("Flies(Animal3)"))
animals_kb.tell(expr("LaysEggs(Animal3)"))
animals_kb.tell(expr("GoodFlyer(Animal3)"))

animals_kb.tell(expr("HasFeathers(Animal4)"))
animals_kb.tell(expr("DoesNotFly(Animal4)"))
animals_kb.tell(expr("Swims(Animal4)"))
animals_kb.tell(expr("BlackAndWhite(Animal4)"))


In [15]:
list(fol_bc_ask(animals_kb, expr('Mammal(y)')))

[{v_16: y, y: Animal1}, {v_16: y, y: Animal2}]

In [16]:
list(fol_bc_ask(animals_kb, expr('Carnivore(y)')))

[{v_49: y, v_66: y, y: Animal1}, {v_49: y, v_66: y, y: Animal2}]

In [11]:
list(fol_bc_ask(animals_kb, expr('Zebra(y)')))

[{v_326: y, v_333: y, v_350: y, y: Animal2}]

In [ ]:
list(fol_fc_ask(animals_kb, expr('Cheetah(y)')))

In [ ]:
list(fol_fc_ask(animals_kb, expr('Tiger(y)')))

In [ ]:
list(fol_fc_ask(animals_kb, expr('Giraffe(y)')))

In [10]:
list(fol_bc_ask(animals_kb, expr('Bird(y)')))

[{v_257: y, y: Animal4}, {v_273: y, y: Animal3}]

In [ ]:
list(fol_bc_ask(animals_kb, expr('Ostrich(y)')))

In [17]:
list(fol_bc_ask(animals_kb, expr('Penguin(y)')))

[{v_223: y, v_226: y, y: Animal4}]

In [ ]:
list(fol_bc_ask(animals_kb, expr('Albatross(y)')))